In [43]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome('./chromedriver')

rating_df = pd.DataFrame()
restaurant_df = pd.DataFrame()

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    lists = ['종로구 맛집', '중구 맛집', '용산구 맛집', '성동구 맛집', '광진구 맛집', '동대문구 맛집', '중랑구 맛집', '성북구 맛집', '강북구 맛집']
#     search('종로구 스톤힐')
    search('종로구 스태픽스')

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
#     driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER) # 더보기
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 전체 페이지
    while True:
        try:
            # 2~ 5페이지 읽기
            for i in range(2, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list
                
                crawling(place, place_lists)
                
                # 다음 페이지 넘기기
                if i==5:
                    driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)

        except ElementNotInteractableException:
            print('end page')
            break
#         finally:
#             search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    
    global restaurant_df

    while_flag = False
    for i, place in enumerate(place_lists):

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        place_local = place.select('.info_item > .addr > .lot_number')[0].text
        place_category = place.select('.head_item > .subcategory')[0].text
        place_detail = place.select('.info_item > .contact> .moreview')[0].get('href') # place detail
        
        row = {"ItemID":place_name, "address": place_address, "local" : place_local, "category": place_category}
        row = pd.DataFrame(row, index=[1])
        restaurant_df = restaurant_df.append(row, ignore_index=True)
        
        driver.execute_script('window.open("about:blank", "_blank");')
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(place_detail) # 상세정보 탭으로 변환
        sleep(1)
        
        print('####', place_name)

        # 첫 페이지
        extract_review(place_name) # 리뷰 추출

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            if page_num > 5:
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page')) #페이지 수 찾기
                
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
                
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver, rating_df

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
            timestamp = review.select(' div > span.time_write') #시간정보
            
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
#                 print(val)                      
                
                try:
                    row = {"ItemID":place_name, "UserID":user_id[0].get('data-userid'), "review" : comment[0].text,
                           "Rating":rating[0].text.replace('점', ''), "Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row, ignore_index=True)
            
                except:
                    row = {"ItemID":place_name, "UserID":None, "review" : None,
                           "Rating": None, "Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                
                
    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

#### 스태픽스
no review in crawling
end page
finish


In [44]:
rating_df

,ItemID,UserID,review,Rating,Timestamp
0,스태픽스,fsigi1,가는길이 힘들었지만 직원분들 너무 친절하시고 얼그레이라떼 너무 맛있게 잘마셨어요 :),5,2021.08.04.
1,스태픽스,1g0qud9,이영자가 갔다온 곳,5,2021.07.28.
2,스태픽스,b1gfce,정원이 정말 예뻐요! 평일이어서 그런지 사람은 그리 많지 않았고 오히려 여유로워서 ...,5,2021.07.27.
3,스태픽스,fmdpa4,그저 입지 덕분.. 사람 진짜 많다,2,2021.07.25.
4,스태픽스,1ls8n9a,,5,2021.07.18.
5,스태픽스,14e5ibh,인싸들 카페.. 너무 붐벼요,3,2021.07.01.
6,스태픽스,1ccgs6u,사람이 왜이렇게 많은걸까요,2,2021.06.28.
7,스태픽스,badsqi,"다들 비슷한 후기를 남긴것 같네요. 가운데 은행나무가 멋지지만 일단, 커피맛이 분위...",2,2021.06.27.
8,스태픽스,1jf7kme,"""인스타 망하면 없어질 카페"" -커피 너무 맛없어서 족욕한 물 마시는게 더 입맛돋...",2,2021.06.25.
9,스태픽스,n031tb,평일에 갔는데도 사람많구요 커피맛없구요 비싸요 +불친절이요,2,2021.06.25.
